In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
cd gdrive/My\ Drive/NNFL\ Project/

/content/gdrive/My Drive/NNFL Project


In [0]:
ls

'AllScratch -  BEST.ipynb'   im1.png             imagenp.npy
 AllScratch.ipynb            im2.png             label_encoder.pickle
 AllScratchV2.ipynb          im3.png             Training/
 checkpointsV2/              im4.png             Training.zip
 embedding_matrix.pickle     im5.png             word_index.pickle
 GGAllScratchV2.ipynb        im6.png
 glove.6B.200d.txt           image_dict.pickle


### CREATING IMAGE PRE-MODEL (4096 OUTPUT) ###

In [0]:
# from keras.applications.vgg16 import VGG16

# base_model = VGG16()
# # print(base_model.summary())

# # get output of penultimate layer
# from keras.models import Model
# model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [0]:
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
# from keras.applications.vgg16 import preprocess_input

# # load an image from file
# image = load_img('Coffee-Mug.jpg', target_size=(224, 224))
# # convert the image pixels to a numpy array
# image = img_to_array(image)
# # reshape data for the model
# image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# # prepare the image for the VGG model
# image = preprocess_input(image)

# # predict the probability across all output classes
# yhat = model.predict(image)
# print(yhat.shape)

### MAKE IMAGE DATA DICT ###
##### key: image name
##### value: vector of shape [1,4096], output of VGGNet

In [0]:
import os
import cv2
import pickle
import numpy as np


# imagedictv2: Images of size 100x100

if(os.path.isfile('image_dict.pickle')):
    with open('image_dict.pickle', 'rb') as handle:
        image_dict = pickle.load(handle)
        
# else:
#   folder = 'Training/images/'

#   image_dict_V2 = {}        

#   skipped = []
#   i=0

#   for filename in os.listdir(folder):
#       if(i%50 == 0):
#           print("{} images processed".format(i))
#       i+=1
#       if filename in image_dict_V2:
#           # print ("already in dict - moving on")
#           continue
#       try:
#           # load an image from file
#           image = cv2.imread(os.path.join(folder, filename))
#       except:
#           print("Error reading file: {}!!!".format(filename))
#           skipped.append(filename)
#           continue
#       if image is not None:
#           resized_image = cv2.resize(image, (100, 100)) 
#           image_dict_V2[filename] = resized_image
#       else:
#           skipped.append(filename)

#   print("{} files skipped:".format(len(skipped)))
#   for f in skipped:
#       print("    {}".format(f))
#   print("dict created")


#   print("Saving image_dict_V2.pickle")
#   with open('image_dict_V2.pickle', 'wb') as handle:
#       pickle.dump(image_dict_V2, handle, protocol=pickle.HIGHEST_PROTOCOL)
#       print('image_dict_V2.pickle saved')



### CREATE QUESTION DATA ###

In [0]:
# create 2 lists
# one stores image ids
# one stores question

imageNamesX  = []
questionsNLX = []
answers = []

import json
QAs = json.load(open("Training/Quest_Answers.json", 'r'))['quest_answers']

for QA in QAs:
    img_name = QA["Image"]+".png"
    ques = QA["Question"]
    ans = QA["Answer"]
    
    if img_name not in image_dict:
        print("Skipping {} - not found in dict".format(img))
        continue
    
    imageNamesX.append(img_name)
    questionsNLX.append(ques)
    answers.append(ans)

In [0]:
### NOW do word embeddings for questions
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 80
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(questionsNLX)
questionsX = tokenizer.texts_to_sequences(questionsNLX)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 200
questionsX = pad_sequences(questionsX, maxlen=max_length_of_text)

print("Saving word_index.pickle")
import pickle
with open('word_index.pickle', 'wb') as handle:
    pickle.dump(word_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Saved.")

Found 80 unique tokens.
Saving word_index.pickle
Saved.


In [0]:
# vector embeddings

embeddings_index = {}
    
EMBEDDING_DIM = 200

embedding_matrix = None
if(os.path.isfile('embedding_matrix.pickle')):
    print(">> Embedding Matrix Pickle found...")
    with open('embedding_matrix.pickle', 'rb') as handle:
        embedding_matrix = pickle.load(handle)
    print(">>> loaded!")
else:
    f = open('glove.6B.200d.txt', encoding="utf8")

    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    embedding_matrix = np.zeros((len(word_index)+1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
            
    with open('embedding_matrix.pickle', 'wb') as handle:
        pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('embedding_matrix.pickle saved')

>> Embedding Matrix Pickle found...
>>> loaded!


In [0]:
embedding_matrix.shape

(81, 200)

In [0]:
# One hot encode answers

## ONE HOT
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

data = answers
values = array(data)
# print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

Y = onehot_encoded

print("Saving label_encoder.pickle")
import pickle
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('label_encoder.pickle saved')

# print(onehot_encoded)
# invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# print(inverted)

def decode_predictions(label_encoder, predictions):
    texts = []
    for p in predictions:
        text = label_encoder.inverse_transform(argmax(p))
        texts.append(text)
    return texts

label_encoder.classes_

Saving label_encoder.pickle
label_encoder.pickle saved


array(['0', '1', '2', '3', '4', '5', '6', '7', '8', 'False', 'True',
       'blue', 'brown', 'cube', 'cyan', 'cylinder', 'gray', 'green',
       'large', 'metal', 'purple', 'red', 'rubber', 'small', 'sphere',
       'yellow'], dtype='<U21')

In [0]:
# Split quesionsX
from sklearn.model_selection import train_test_split
imageNamesX_train, imageNamesX_test, questionsX_train, questionsX_test, Y_train, Y_test = train_test_split(imageNamesX, questionsX, Y, test_size=0.1, random_state=42)

### Data generator ###

In [0]:
# def generator(image_dict, img_names, questions, labels, batch_size):
#     # Create empty arrays to contain batch of features and labels
    
# #     batch_features = np.zeros((batch_size, 64, 64, 3))
# #     batch_labels = np.zeros((batch_size,1))
    
#     q_ptr = 0
#     batch_features = []
#     batch_labels = []
#     while True:
#         for i in range(batch_size):
#             if q_ptr == len(questions):
#                 q_ptr = 0
#             index = q_ptr
#             # print(imageNamesX[q_ptr].shape)
#             # print(questionsX[q_ptr])
#             batch_features.append(np.concatenate((image_dict[img_names[index]], questions[index]), axis = None))
#             batch_labels.append(Y[index])
#             q_ptr+=1
#         yield np.array(batch_features), np.array(batch_labels)

In [0]:
# A, B = generator(image_dict, imageNamesX, questionsX, Y, 10)
# print(A)
# print(B)

In [0]:
def generator(image_dict, img_names, questions, labels, batch_size):
    
    q_ptr = 0
    while True:
        image_inp = []
        q_inp = []
        batch_labels = []
        for i in range(batch_size):
            if q_ptr == len(questions):
                q_ptr = 0
            index = q_ptr
#             import random
#             index= random.randint(0, len(questions)-1)
            # print(imageNamesX[q_ptr].shape)
            # print(questionsX[q_ptr])
            image_inp.append(image_dict[img_names[index]])
            q_inp.append(questions[index])
            batch_labels.append(labels[index])
            q_ptr+=1
            
        yield [np.array(image_inp), np.array(q_inp)], np.array(batch_labels)

In [0]:
# def test_generator(image_dict, img_names, questions, labels, batch_size):
    
#     q_ptr = 0
#     while True:
#         image_inp = []
#         q_inp = []
#         batch_labels = []
#         for i in range(batch_size):
#             if q_ptr == len(questions):
#                 q_ptr = 0
#             index = q_ptr
# #             import random
# #             index= random.randint(0, len(questions)-1)
#             # print(imageNamesX[q_ptr].shape)
#             # print(questionsX[q_ptr])
#             image_inp.append(image_dict[img_names[index]])
#             q_inp.append(questions[index])
#             batch_labels.append(labels[index])
#             q_ptr+=1
#         yield [np.array(image_inp), np.array(q_inp)], np.array(batch_labels)

In [0]:
# from keras.layers import Conv2D, MaxPooling2D, Flatten
# from keras.layers import Input, LSTM, Embedding, Dense
# from keras.models import Model, Sequential
# import keras

# # Define CNN for Image Input
# vision_model = Sequential()
# vision_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
# vision_model.add(Conv2D(64, (3, 3), activation='relu'))
# vision_model.add(MaxPooling2D((2, 2)))
# vision_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# vision_model.add(Conv2D(128, (3, 3), activation='relu'))
# vision_model.add(MaxPooling2D((2, 2)))
# vision_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
# vision_model.add(Conv2D(256, (3, 3), activation='relu'))
# vision_model.add(Conv2D(256, (3, 3), activation='relu'))
# vision_model.add(MaxPooling2D((2, 2)))
# vision_model.add(Flatten())

# image_input = Input(shape=(224, 224, 3))
# encoded_image = vision_model(image_input)

# # Define RNN for language input
# question_input = Input(shape=(100,), dtype='int32')
# embedded_question = Embedding(input_dim=10000, output_dim=256, input_length=100)(question_input)
# encoded_question = LSTM(256)(embedded_question)

# # Combine CNN and RNN to create the final model
# merged = keras.layers.concatenate([encoded_question, encoded_image])
# output = Dense(1000, activation='softmax')(merged)
# vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# print(vqa_model.summary())

# vqa_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
#         metrics=['accuracy'])

# vqa_model.fit_generator(generator2(image_dict, imageNamesX, questionsX, Y, 10), samples_per_epoch=50, nb_epoch=1)

### MODELS ###

In [0]:
#BEST
def vqa_model0(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    import vqa_model0
    model = vqa_model0.get_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes)
    return model

In [0]:
def vqa_model1(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    import vqa_model1
    model = vqa_model1.get_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes)
    return model

In [0]:
#ANet Experiment
def vqa_model3(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    import vqa_model3
    model = vqa_model3.get_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes)
    return model

In [0]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Dropout, LSTM, Flatten, Embedding, Multiply, Concatenate
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import keras

def vqa_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    
    print("Creating image model...")
    img_model = Sequential()
    img_model.add(Dense(1024, input_dim=4096, activation='tanh'))

    image_input = Input(shape=(4096, ))
    encoded_image = img_model(image_input)

    print("Creating text model...")
    txt_model = Sequential()
    txt_model.add(Embedding(num_words, embedding_dim, 
        weights=[embedding_matrix], input_length=seq_length, trainable=False))
    txt_model.add(LSTM(units=512, return_sequences=True, input_shape=(seq_length, embedding_dim)))
    txt_model.add(Dropout(dropout_rate))
    txt_model.add(LSTM(units=512, return_sequences=False))
    txt_model.add(Dropout(dropout_rate))
    txt_model.add(Dense(1024, activation='tanh'))
    
    question_input = Input(shape=(EMBEDDING_DIM, ), dtype='int32')
    embedded_question = txt_model(question_input)
    
    print("Merging final model...")
    merged = keras.layers.concatenate([encoded_image, embedded_question])
    d1  = Dense(1000, activation='softmax')(merged)
    dp1 = Dropout(dropout_rate)(d1)
    d2  = Dense(1000, activation='tanh')(dp1)
    dp2 = Dropout(dropout_rate)(d2)
    output  = Dense(num_classes, activation='softmax')(dp2)
    
    vqa_model = Model(inputs=[image_input, question_input], outputs=output)
    
    
#     fc_model = Sequential()
#     # fc_model.add(Merge([vgg_model, lstm_model], mode='mul'))
#     fc_model.add(Concatenate([img_model, txt_model]))
#     fc_model.add(Dropout(dropout_rate))
#     fc_model.add(Dense(1000, activation='tanh'))
#     fc_model.add(Dropout(dropout_rate))
#     fc_model.add(Dense(num_classes, activation='softmax'))
    
    vqa_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return vqa_model

In [0]:
dropout_rate=0.5
num_classes=26
model_weights_filename = "weightsV2.bkp"
ckpt_model_weights_filename = "checkpointsV2/checkP.cp"

model = None
if os.path.exists(ckpt_model_weights_filename):
    print("Loading model: {}".format(ckpt_model_weights_filename))
    from keras.models import load_model
    model = load_model(ckpt_model_weights_filename)
else:
    model = vqa_model(embedding_matrix, num_words, EMBEDDING_DIM, max_length_of_text, dropout_rate, num_classes)
    if os.path.exists(model_weights_filename):
        print ("Loading Weights...")
        model.load_weights(model_weights_filename)
    
# print(model.summary())

from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath=ckpt_model_weights_filename, verbose=1)

batch_size = 10
model.fit_generator(
    generator(image_dict, imageNamesX_train, questionsX_train, Y_train, batch_size),
    validation_data=generator(image_dict, imageNamesX_test, questionsX_test, Y_test, batch_size),
    validation_steps=len(Y_test)/(batch_size-1), steps_per_epoch=len(Y_train)/(batch_size-1), nb_epoch=1, callbacks=[checkpointer])

Creating image model...
Creating text model...
Merging final model...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., validation_steps=1500.22222..., steps_per_epoch=13502.0, callbacks=[<keras.ca..., epochs=1)`


Epoch 1/1
  728/13502 [>.............................] - ETA: 3:42:08 - loss: 2.6684 - acc: 0.2055

KeyboardInterrupt: ignored

### EXPERIMENTS ###

In [0]:
# # EXPERIMENT
# from sklearn.preprocessing import MultiLabelBinarizer
# labels = ["hello", "world", "hello", "cat"]
# mlb = MultiLabelBinarizer()
# labels = mlb.fit_transform(labels)

In [0]:
# ## ONE HOT
# from numpy import array
# from numpy import argmax
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# # define example
# data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
# values = array(data)
# print(values)
# # integer encode
# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)
# # binary encode
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
# # # invert first example
# # inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# # print(inverted)

# def decode_prediction(predictions):
#     texts = []
#     for p in predictions:
#         text = label_encoder.inverse_transform(argmax(p))
#         texts.append(text)
#     return texts
    
# asdf = []
# asdf.append([1,0,0])
# asdf.append([0,1,0])
# asdf.append([0,0,1])
    
# print(decode_prediction(asdf))

### PLAY AROUND ###

In [0]:
import pandas as pd
import argparse
from keras.models import load_model
import re
import pickle
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import cv2

# -------VARIABLES------- #
max_length_of_text = 42
# ----------------------- #

def VGG_preprocess_images(images):
    print("> VGG preprocessing images")
    
    from keras.applications.vgg16 import VGG16
    base_model = VGG16()
    # print(base_model.summary())

    # get output of penultimate layer
    from keras.models import Model
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

    print(">> VGG keras model loaded")
    
    from keras.preprocessing.image import load_img
    from keras.preprocessing.image import img_to_array
    from keras.applications.vgg16 import preprocess_input
    
    resized_images = []
    for image in images:
        resized_image = cv2.resize(image, (224, 224)) 
        resized_images.append(resized_image)
    
    resized_images = np.array(resized_images)
    print(resized_images.shape)
        
    # prepare the image for the VGG model
    preprocessed_images = preprocess_input(resized_images)
    
    print(">> Images VGG preprocessed!")

    # predict the probability across all output classes
    results = model.predict(preprocessed_images)
    
    return results
    
def filter_string(str1, str2):
    for c in str2:
        str1 = str1.replace(c, '')
    return str1

def predict(model_path, images, questions, word_index):
    print("> Loading our ML model...")
    model = load_model(model_path)
    print(">> model loaded")
    
    # question is a list of NL questions
    # convert to sequences
    print("> Processing NL questions")
    X = []
    for Q in questions:
        temp = []
        Q = filter_string(Q, '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
        for word in Q.split():
            if(word in word_index):
                temp.append(word_index[word])
        X.append(temp)    
    X = pad_sequences(X, maxlen=max_length_of_text)
    print(">> Done!")
    
#     print("\nimages:")
#     print(images)
    
#     print("\nX:")
#     print(X)
    
    print("> Predicting results")
    predictions = model.predict([np.array(images), np.array(X)])
    return predictions

def decode_predictions(label_encoder, predictions):
    texts = []
    for p in predictions:
        top3 = []
        s = np.argsort(p, axis=-1, kind='quicksort', order=None)
        for i in range (len(s)-1, len(s)-6, -1):
            text = label_encoder.inverse_transform(s[i])
            top3.append(text)
        texts.append(top3)
    return texts

def results_to_text(label_encoder, results):
    print("> Converting model output to texts")
    texts = decode_predictions(label_encoder, results)
    return texts

def playground():
    #make questions and images
    questions = ["How many spheres are present?",
                 "What color is the large metallic ball",
                 "How many cylinders are present",
                 "What color is the large metallic ball that is in front of the metallic ball on the left side of the cylinder in front of the blue sphere?",
                 "Are there more small metallic objects on the left side of the big purple metallic object than gray balls that are to the right of the tiny gray rubber ball?",
                 "How many other objects are there of the same shape as the small blue matte thing?"]
    image_names = ["im1.png",
                   "im2.png",
                   "im3.png",
                   "im4.png",
                   "im5.png",
                   "im6.png"]

    images = []
    for image_name in image_names:
        image = cv2.imread(image_name)
        resized_image = cv2.resize(image, (224, 224)) 
        images.append(resized_image)
        
    preprocessed_images = VGG_preprocess_images(images)

    model_path = "checkpoints/checkP.cp"

    word_index = None
    with open('word_index.pickle', 'rb') as handle:
        word_index = pickle.load(handle)
    if word_index is not None:
        print("\n> word_index.pickle loaded.")

        results = predict(model_path, preprocessed_images, questions, word_index)
        print(results)
        
        label_encoder = None
        with open('label_encoder.pickle', 'rb') as handle:
            label_encoder = pickle.load(handle)
        texts = results_to_text(label_encoder, results)
    
        for i in range(0, len(questions)):
            img = cv2.imread(image_names[i])
            
            from matplotlib import pyplot as plt
            plt.imshow(img)
            plt.show()
            
            Q = questions[i]
            print(Q)
            A = texts[i]
            print(A)
            print()

    #     print("\nSaving solution.csv...")
    #     new_df = pd.DataFrame({"Index": [i+1 for i in range (len(predictions))],
    #                             "Sentiment": predictions})
    #     new_df.to_csv("solution.csv", index = False)
    #     print("Saved.")
    else:
        print("Sorry, word_index.pickle not found!")
        
# print("Freeing GPU Memory...")
# # from numba import cuda
# # cuda.select_device(0)
# # cuda.close()

# # from keras import backend as K
# # K.clear_session()
# print("Freed!\n")
        
playground()

In [0]:
# from matplotlib import pyplot as plt
# import cv2

# img1 = cv2.imread("im1.png")
# img2 = cv2.imread("im2.png")

# plt.imshow(img1)
# plt.show()
# plt.imshow(img2)
# plt.show()